In [1]:
import numpy as np
import pandas as pd

In [2]:
data1 = pd.read_excel('超市.xlsx')
data1.head(1)

,行 ID,订单 ID,订单日期,发货日期,邮寄方式,客户 ID,客户名称,细分,城市,省/自治区,国家,地区,产品 ID,类别,子类别,产品名称,销售额,数量,折扣,利润
0,1,US-2018-1357144,2018-04-27,2018-04-29,二级,曾惠-14485,曾惠,公司,杭州,浙江,中国,华东,办公用-用品-10002717,办公用品,用品,"Fiskars 剪刀, 蓝色",129.696,2,0.4,-60.704


### 需求：分析一下各省的销售额分布情况

###### 取数

In [3]:
data2 = data1.groupby('省/自治区')['销售额'].sum()
# 还是那理由
data2 = pd.DataFrame(data2)
data2.head(1)

,销售额
省/自治区,
上海,582450.568


In [4]:
data2.head(1)

,销售额
省/自治区,
上海,582450.568


In [8]:
from pyecharts import options as opts
from pyecharts.charts import Geo
from pyecharts.faker import Faker

# 第一个值
x = data2.index.tolist()
# 第二个值
y = data2['销售额'].tolist()

c = (
    Geo()
    .add_schema(maptype="china")     # 👇 添加到这
    .add("geo", [list(z) for z in zip(x, y)])
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        visualmap_opts=opts.VisualMapOpts(), title_opts=opts.TitleOpts(title="Geo-基本示例")
    )
)
c.render_notebook()

In [10]:
from pyecharts import options as opts
from pyecharts.charts import Geo
from pyecharts.faker import Faker
from pyecharts.globals import ChartType

c = (
    Geo()
    .add_schema(maptype="china")
    .add(
        "geo",
        [list(z) for z in zip(x, y)],
        type_=ChartType.HEATMAP,
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        visualmap_opts=opts.VisualMapOpts(),
        title_opts=opts.TitleOpts(title="Geo-HeatMap"),
    )
)
c.render_notebook()

#### 雷达图

#### 需求：查看一下各地区细分的销售额

In [13]:
data3 = data1.groupby(['地区','细分'])['销售额'].sum()
data3 = pd.DataFrame(data3).unstack()
data3 

销售额                         
细分           公司        小型企业          消费者
地区                                      
东北   834842.827  522739.035  1323985.607
中南  1335665.324  743813.007  2057936.762
华东  1454715.808  942432.372  2287358.262
华北   804769.469  422100.987  1220430.561
西北   253458.184  103523.308   458058.104
西南   469341.684  156479.932   677302.892

In [25]:
# 取三个数据
data_1 = [data3['销售额']['公司'].tolist()]
data_2 = [data3['销售额']['小型企业'].tolist()]
data_3 = [data3['销售额']['消费者'].tolist()]
data_3

[[1323985.607, 2057936.762, 2287358.262, 1220430.561, 458058.104, 677302.892]]

In [29]:
import pyecharts.options as opts
from pyecharts.charts import Radar

"""
Gallery 使用 pyecharts 1.1.0
参考地址: https://echarts.apache.org/examples/editor.html?c=radar

目前无法实现的功能:

1、雷达图周围的图例的 textStyle 暂时无法设置背景颜色
"""
v1 = data_1
v2 = data_2
v3 = data_3

c = (
    Radar(init_opts=opts.InitOpts(bg_color="#CCCCCC"))
    .add_schema(
        schema=[
            opts.RadarIndicatorItem(name="东北", max_=2000000),
            opts.RadarIndicatorItem(name="中南", max_=2000000),
            opts.RadarIndicatorItem(name="华东", max_=2000000),
            opts.RadarIndicatorItem(name="华北", max_=2000000),
            opts.RadarIndicatorItem(name="西北", max_=2000000),
            opts.RadarIndicatorItem(name="西南", max_=2000000),
        ],
        splitarea_opt=opts.SplitAreaOpts(
            is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
        ),
        textstyle_opts=opts.TextStyleOpts(color="#fff"),
    )
    .add(
        series_name="预算分配（Allocated Budget）",
        data=v1,
        linestyle_opts=opts.LineStyleOpts(color="#CD0000"),
    )
    .add(
        series_name="实际开销（Actual Spending）",
        data=v2,
        linestyle_opts=opts.LineStyleOpts(color="#5CACEE"),
    )
    .add(
        series_name="分配（Allocated Budget）",
        data=v3,
        linestyle_opts=opts.LineStyleOpts(color="#CD0000"),
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="基础雷达图"), legend_opts=opts.LegendOpts()
    )
)
c.render_notebook()


### 漏斗图

#### 分析用户行为

In [33]:
data4 = pd.read_excel('./漏斗.xlsx')
data4

,用户主要行为,用户数
0,DAU,700000
1,登录,500000
2,加入购物车,300000
3,生成订单,210000
4,支付,150000
5,完成交易,140000


In [37]:
from pyecharts import options as opts
from pyecharts.charts import Funnel
from pyecharts.faker import Faker

# 取数
x = data4['用户主要行为']
y = data4['用户数']

c = (
    Funnel()                       
    .add(
        "商品",              #  👇 放这
        [list(z) for z in zip(x,y)],
        # 👇 这个使 标签 放 内部
        label_opts = opts.LabelOpts(position = "inside")
    ) 
    .set_global_opts(title_opts=opts.TitleOpts(title="分析用户行为"))
)
c.render_notebook()

#### 象形图

#### 需求：不同地区销售额对比

In [41]:
df1 = data1.groupby('地区')['销售额'].sum()
df1 = pd.DataFrame(df1)
df1

,销售额
地区,
东北,2681567.469
中南,4137415.093
华东,4684506.442
华北,2447301.017
西北,815039.596
西南,1303124.508


In [42]:
# 取数
location = df1.index.to_list()
values =df1['销售额'].to_list()

[2681567.469, 4137415.093, 4684506.442, 2447301.017, 815039.596, 1303124.508]

In [44]:
from pyecharts import options as opts
from pyecharts.charts import PictorialBar
from pyecharts.globals import SymbolType

c = (
PictorialBar()  # 👇第一个参数
    .add_xaxis(location)
    .add_yaxis(
        "",
        # 👇 第二个参数
        values,
        label_opts=opts.LabelOpts(is_show=False),
        symbol_size=18,
        symbol_repeat="fixed",
        is_symbol_clip=True,
        symbol=SymbolType.ROUND_RECT
    )                     # 👆 这个是形状
    .reversal_axis()
    .set_global_opts(
        title_opts=opts.TitleOpts(title="各地区销售额分布"),
        xaxis_opts=opts.AxisOpts(is_show=False),
        yaxis_opts=opts.AxisOpts()
    )
)
c.render_notebook()